In [ ]:
!pip install tensorflow
!pip install keras
!pip install matplotlib
!pip install numpy
!pip install pandas
!pip install glob
!pip install random
!pip install matplotlib.patches
!pip install lxml

In [ ]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import glob
import random
from matplotlib.patches import Rectangle
from lxml import etree
%matplotlib inline

### 1. Data Preprocessing

#### 1.1 Image_Label_Preview

In [ ]:
import os
os.listdir('/content/drive/MyDrive/Colab Notebooks/ai프로젝트/input/images/images')

In [ ]:
os.listdir('/content/drive/MyDrive/Colab Notebooks/ai프로젝트/input/label/label')

#### 1.2 Load Data

In [ ]:
image_path = glob.glob('/content/drive/MyDrive/Colab Notebooks/ai프로젝트/input/images/crease/*.jpg')
len(image_path)

In [ ]:
xmls_path = glob.glob('/content/drive/MyDrive/Colab Notebooks/ai프로젝트/input/images/crease/*.xml')
len(xmls_path)

In [ ]:
import glob

 #glob 모듈을 사용하여 특정 경로 내에서 .jpg 파일을 모두 찾음
# image_path = glob.glob('/content/drive/MyDrive/project/steel_data/images/*/*.jpg')
image_path = glob.glob('/content/drive/MyDrive/Colab Notebooks/ai프로젝트/input/images/crease/*.jpg')
len(image_path)

img_names = [img.split('/')[-1].split('.jpg')[0] for img in image_path]
print(img_names)
 #glob 모듈을 사용하여 특정 경로 내에서 .xml 파일을 모두 찾음
# xmls_path = glob.glob('/content/drive/MyDrive/project/steel_data/label/label/*.xml')
xmls_path = glob.glob('/content/drive/MyDrive/Colab Notebooks/ai프로젝트/input/images/crease/*.xml')
# print(xmls_path[0])
xml_names = [xml.split('/')[-1].split('.xml')[0] for xml in xmls_path]
print(xml_names)
#XML 파일 이름만 추출
#p.split('/')[-1] 분리된 리스트에서 마지막 요소를 선택
#.split('.')[0]: 파일 이름을 . 기준으로 분리하여 확장자를 제거하고, 순수한 파일 이름만 추출
xmls_train = [p.split('/')[-1].split('.')[0] for p in xmls_path]

imgs_train = [img for img in image_path if img.split('/')[-1].split('.jpg')[0] in xml_names]

xmls_path = [xml for xml in xmls_path if xml.split('/')[-1].split('.xml')[0] in img_names]


In [ ]:
len(xmls_path),len(imgs_train)

In [ ]:
#labels names
names = [x.split("/")[-2] for x in imgs_train]
names[:3]

In [ ]:
names = pd.DataFrame(names,columns=['Types'])
names

In [ ]:
#onehot for mutiple classes
from sklearn.preprocessing import LabelBinarizer

Class = names['Types'].unique()
Class_dict = dict(zip(Class, range(1,len(Class)+1)))
names['str'] = names['Types'].apply(lambda x: Class_dict[x])
lb = LabelBinarizer()
lb.fit(list(Class_dict.values()))
transformed_labels = lb.transform(names['str'])
y_bin_labels = []

for i in range(transformed_labels.shape[1]):
    y_bin_labels.append('str' + str(i))
    names['str' + str(i)] = transformed_labels[:, i]

In [ ]:
Class_dict

In [ ]:
names.drop('str',axis=1,inplace=True)
names.drop('Types',axis=1,inplace=True)
names.head()

#### 1.3 Extraction & Input pipe

In [ ]:
#analysis rectangular box value in xmls
def to_labels(path):
    xml = open('{}'.format(path)).read()                         #read xml in path
    sel = etree.HTML(xml)
    width = int(sel.xpath('//size/width/text()')[0])     #extract the width/height
    height = int(sel.xpath('//size/height/text()')[0])    #extract the x,y value
    xmin = int(sel.xpath('//bndbox/xmin/text()')[0])
    xmax = int(sel.xpath('//bndbox/xmax/text()')[0])
    ymin = int(sel.xpath('//bndbox/ymin/text()')[0])
    ymax = int(sel.xpath('//bndbox/ymax/text()')[0])
    return [xmin/width, ymin/height, xmax/width, ymax/height]   #return the four relative points

In [ ]:
#set value to labels
labels = [to_labels(path) for path in xmls_path]
labels[:3]

In [ ]:
#set four labels as outputs
out1,out2,out3,out4 = list(zip(*labels))
#convert to np.array
out1 = np.array(out1)
out2 = np.array(out2)
out3 = np.array(out3)
out4 = np.array(out4)
label = np.array(names.values)

In [ ]:
#label to tf.data
label_datasets = tf.data.Dataset.from_tensor_slices((out1,out2,out3,out4,label))
label_datasets

In [ ]:
#def load_image function
def load_image(path):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image,3)
    image = tf.image.resize(image,[224,224])
    image = tf.cast(image/127.5-1,tf.float32)
    return image

In [ ]:
#build dataset
dataset = tf.data.Dataset.from_tensor_slices(imgs_train)
dataset = dataset.map(load_image)

In [ ]:
dataset_label = tf.data.Dataset.zip((dataset,label_datasets))

In [ ]:
#batch constant
BATCH_SIZE = 16
AUTO = tf.data.experimental.AUTOTUNE

In [ ]:
#batch extraction and shuffle
dataset_label = dataset_label.repeat().shuffle(500).batch(BATCH_SIZE)
dataset_label = dataset_label.prefetch(AUTO)

In [ ]:
#Split dataset
test_count = int(len(imgs_train)*0.2)
train_count = len(imgs_train) - test_count
test_count,train_count

In [ ]:
#Split dataset
test_count = int(len(imgs_train)*0.2)
train_count = len(imgs_train) - test_count
# test_count,train_count

train_dataset = dataset_label.skip(test_count)
test_dataset = dataset_label.take(test_count)

# train_dataset

In [ ]:
species_dict = {v:k for k,v in Class_dict.items()}

In [ ]:
#check from train_data
for img, label in train_dataset.take(1):
    plt.imshow(keras.preprocessing.image.array_to_img(img[0]))
    out1,out2,out3,out4,out5 = label
    xmin,ymin,xmax,ymax = out1[0].numpy()*224,out2[0].numpy()*224,out3[0].numpy()*224,out4[0].numpy()*224
    rect = Rectangle((xmin,ymin),(xmax-xmin),(ymax-ymin),fill=False,color='r')
    ax = plt.gca()
    ax.axes.add_patch(rect)
    pred_imglist = []
    pred_imglist.append(species_dict[np.argmax(out5[0])+1])
    plt.title(pred_imglist)
    plt.show()

### 2. Model Building

#### 2.1 The VGG16 model

In [ ]:
#Convolution based
conv = keras.applications.xception.Xception(weights='imagenet',
                                            include_top=False,
                                            input_shape=(224,224,3),
                                            pooling='avg')

In [ ]:
#open trainable
conv.trainable = True

In [ ]:
#define Conv + FC structure
inputs = keras.Input(shape=(224,224,3))
x = conv(inputs)
x1 = keras.layers.Dense(1024,activation='relu')(x)
x1 = keras.layers.Dense(512,activation='relu')(x1)


out1 = keras.layers.Dense(1,name='out1')(x1)
out2 = keras.layers.Dense(1,name='out2')(x1)
out3 = keras.layers.Dense(1,name='out3')(x1)
out4 = keras.layers.Dense(1,name='out4')(x1)

x2 = keras.layers.Dense(1024,activation='relu')(x)
x2 = keras.layers.Dropout(0.5)(x2)
x2 = keras.layers.Dense(512,activation='relu')(x2)
out_class = keras.layers.Dense(10,activation='softmax',name='out_item')(x2)

out = [out1,out2,out3,out4,out_class]

model = keras.models.Model(inputs=inputs,outputs=out)
model.summary()

In [ ]:
from tensorflow.keras.utils import to_categorical

# Class_dict 정의
Class_dict = {
    'crease': 1,
    'rolled_pit': 2,
    'welding_line': 3,
    'water_spot': 4,
    'silk_spot': 5,
    'punching': 6,
    'oil_spot': 7,
    'waist_folding': 8,
    'inclusion': 9,
    'crescent_gap': 10
}

# 이미지에 대한 클래스 레이블을 Class_dict의 정수값으로 변환하여 valid_labels에 저장
image_classes = ['crease', 'welding_line', 'oil_spot', 'crease']

valid_labels = [Class_dict[cls] for cls in image_classes]

print(valid_labels)  # 정수형 레이블

# 정답 레이블을 원핫 인코딩하지 않고 정수형 레이블 그대로 사용
# 모델 컴파일 (각 출력에 맞는 loss와 metrics 제공)
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001),  # 초기 학습률 설정
              loss={
                  'out1': 'mse',
                  'out2': 'mse',
                  'out3': 'mse',
                  'out4': 'mse',
                  'out_item': 'sparse_categorical_crossentropy'  # 정수형 레이블 사용
              },
              metrics={
                  'out1': 'mae',
                  'out2': 'mae',
                  'out3': 'mae',
                  'out4': 'mae',
                  'out_item': 'accuracy'  # 정확도 사용
              })

# 학습률 감소 콜백 설정
lr_reduce = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', patience=6, factor=0.5, min_lr=1e-8)

# 모델 학습
history = model.fit(
    train_dataset,  # 학습 데이터셋
    steps_per_epoch=train_count // BATCH_SIZE,  # 학습 데이터에서 스텝 수 계산
    epochs=30,  # 에포크 수 설정
    validation_data=test_dataset,  # 검증 데이터셋
    validation_steps=test_count // BATCH_SIZE,  # 검증 데이터에서 스텝 수 계산
    callbacks=[lr_reduce]  # 학습률 감소 콜백 추가
)


In [ ]:
#training visualization
def plot_history(history):
    hist = pd.DataFrame(history.history)
    hist['epoch']=history.epoch

    plt.figure()
    plt.xlabel('Epoch')
    plt.ylabel('MSE')
    plt.plot(hist['epoch'],hist['loss'],
            label='Train Loss')
    plt.plot(hist['epoch'],hist['val_loss'],
            label='Val Loss')
    plt.legend()

    plt.figure()
    plt.xlabel('Epoch')
    plt.ylabel('Val_MAE')
    plt.plot(hist['epoch'],hist['val_out1_mae'],
            label='Out1_mae')
    plt.plot(hist['epoch'],hist['val_out2_mae'],
            label='Out2_mae')
    plt.plot(hist['epoch'],hist['val_out3_mae'],
            label='Out3_mae')
    plt.plot(hist['epoch'],hist['val_out4_mae'],
            label='Out4_mae')
    plt.legend()

    # plt.figure()
    # plt.xlabel('Epoch')
    # plt.ylabel('Val_Item_Acc')
    plt.figure()
    plt.xlabel('Epoch')
    plt.ylabel('Val_Item_Acc')
    plt.plot(hist['epoch'], hist['val_out_item_accuracy'], label='Out5_acc')
    plt.legend()

    plt.show()

plot_history(history)

#### 2.3 Model Evaluation

In [ ]:
mae = model.evaluate(test_dataset)

In [ ]:
# print('out1_mae in test:{}'.format(mae[6]))
# print('out2_mae in test:{}'.format(mae[8]))
# print('out3_mae in test:{}'.format(mae[10]))
# print('out4_mae in test:{}'.format(mae[12]))
# print('class_label in test:{}'.format(mae[15]))
print('out1_mae in test:{}'.format(mae[0]))
print('out2_mae in test:{}'.format(mae[1]))
print('out3_mae in test:{}'.format(mae[2]))
print('out4_mae in test:{}'.format(mae[3]))
print('class_label in test:{}'.format(mae[5]))  # out_item_accuracy


In [ ]:
model.save("C:\KDT_SF6\비전ai\model_save/class_location_test_crease.h5")

In [ ]:
species_dict = {v:k for k,v in Class_dict.items()}

In [ ]:
species_dict

In [ ]:
# plt.figure(figsize=(10,24))
# for img,_ in train_dataset.take(1):
#     out1, out2, out3, out4, out5 = model.predict(img)
#     for i in range(3):
#         plt.axis('off')
#         plt.subplot(3, 1, i+1)
#         plt.imshow(keras.preprocessing.image.array_to_img(img[i]))

#         pred_class_index = np.argmax(out5[i]) + 1  # 예측된 클래스 인덱스
#         print(f"Predicted class index: {pred_class_index}")

#         pred_imglist = []
#         # species_dict에서 예측된 클래스 인덱스가 존재하는지 확인
#         if pred_class_index in species_dict:
#             pred_imglist.append(species_dict[pred_class_index])
#         else:
#             pred_imglist.append(f"Unknown class {pred_class_index}")

#         plt.title(pred_imglist)

#         xmin, ymin, xmax, ymax = out1[i] * 224, out2[i] * 224, out3[i] * 224, out4[i] * 224
#         rect = Rectangle((xmin, ymin), (xmax - xmin), (ymax - ymin), fill=False, color='r')
#         ax = plt.gca()
#         ax.axes.add_patch(rect)
plt.figure(figsize=(10,24))

# 모델에서 예측한 출력(out5)이 클래스 예측 결과라고 가정
for img, _ in train_dataset.take(1):
    out1, out2, out3, out4, out5 = model.predict(img)

    # 3개의 이미지를 순회
    for i in range(3):
        plt.axis('off')  # 축 표시 제거
        plt.subplot(3, 1, i+1)

        # 이미지 시각화
        plt.imshow(keras.preprocessing.image.array_to_img(img[i]))

        # 예측된 클래스 인덱스
        pred_class_index = np.argmax(out5[i]) + 1  # species_dict가 1부터 시작한다면

        # 클래스가 항상 species_dict에 존재한다고 가정하고 처리
        pred_imglist = [species_dict[pred_class_index]]

        # 제목 설정
        plt.title(pred_imglist)

        # 바운딩 박스 그리기
        xmin, ymin, xmax, ymax = out1[i] * 224, out2[i] * 224, out3[i] * 224, out4[i] * 224
        rect = Rectangle((xmin, ymin), (xmax - xmin), (ymax - ymin), fill=False, color='r')
        ax = plt.gca()  # 현재의 축 가져오기
        ax.axes.add_patch(rect)

# 시각화한 이미지 출력
plt.show()


In [ ]:
# !pip install tensorflow
# !pip install keras
# !pip install matplotlib
# !pip install numpy
# !pip install pandas
# !pip install glob
# !pip install random
# !pip install matplotlib.patches
# !pip install lxml
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import glob
import random
from matplotlib.patches import Rectangle
from lxml import etree
import os
import glob
%matplotlib inline
결함s = ['crease','crescent_gap','oil_spot','rolled_pit','punching_hole','silk_spot','inclusion','waist folding','water_spot','welding_line']
# 1. 로드 데이터
결함 = 'rolled_pit'
os.listdir('/content/drive/MyDrive/비전ai/archive/images/images/images')
os.listdir('/content/drive/MyDrive/비전ai/archive/label/xml_single_bndbox')
# image_path = glob.glob('/content/drive/MyDrive/비전ai/archive/images/images/images/crease/*.jpg')
# # len(image_path)

# xmls_path = glob.glob('/content/drive/MyDrive/비전ai/archive/label/xml_single_bndbox/*.xml')
# len(xmls_path)
# for i in 결함:
#   try:
  #glob 모듈을 사용하여 특정 경로 내에서 .jpg 파일을 모두 찾음
  # image_path = glob.glob('/content/drive/MyDrive/project/steel_data/images/*/*.jpg')
image_path = glob.glob(f'/content/drive/MyDrive/비전ai/archive/images/images/images/{결함}/*.jpg')
# len(image_path)

img_names = [img.split('/')[-1].split('.jpg')[0] for img in image_path]
print(img_names)
#glob 모듈을 사용하여 특정 경로 내에서 .xml 파일을 모두 찾음
# xmls_path = glob.glob('/content/drive/MyDrive/project/steel_data/label/label/*.xml')
xmls_path = glob.glob('/content/drive/MyDrive/비전ai/archive/label/label/*.xml')
# print(xmls_path[0])
xml_names = [xml.split('/')[-1].split('.xml')[0] for xml in xmls_path]
print(xml_names)
#XML 파일 이름만 추출
#p.split('/')[-1] 분리된 리스트에서 마지막 요소를 선택
#.split('.')[0]: 파일 이름을 . 기준으로 분리하여 확장자를 제거하고, 순수한 파일 이름만 추출
xmls_train = [p.split('/')[-1].split('.')[0] for p in xmls_path]

imgs_train = [img for img in image_path if img.split('/')[-1].split('.jpg')[0] in xml_names]

xmls_path = [xml for xml in xmls_path if xml.split('/')[-1].split('.xml')[0] in img_names]
names = [x.split("/")[-2] for x in imgs_train]
names = pd.DataFrame(names,columns=['Types'])
#onehot for mutiple classes
from sklearn.preprocessing import LabelBinarizer

Class = names['Types'].unique()
Class_dict = dict(zip(Class, range(1,len(Class)+1)))

names['str'] = names['Types'].apply(lambda x: Class_dict[x])

lb = LabelBinarizer()
lb.fit(list(Class_dict.values()))

transformed_labels = lb.transform(names['str'])

y_bin_labels = []

for i in range(transformed_labels.shape[1]):
    y_bin_labels.append('str' + str(i))
    names['str' + str(i)] = transformed_labels[:, i]

names.drop('str',axis=1,inplace=True)
names.drop('Types',axis=1,inplace=True)
# names.head()

#analysis rectangular box value in xmls
def to_labels(path):
    xml = open('{}'.format(path)).read()                         #read xml in path
    sel = etree.HTML(xml)
    width = int(sel.xpath('//size/width/text()')[0])     #extract the width/height
    height = int(sel.xpath('//size/height/text()')[0])    #extract the x,y value
    xmin = int(sel.xpath('//bndbox/xmin/text()')[0])
    xmax = int(sel.xpath('//bndbox/xmax/text()')[0])
    ymin = int(sel.xpath('//bndbox/ymin/text()')[0])
    ymax = int(sel.xpath('//bndbox/ymax/text()')[0])
    return [xmin/width, ymin/height, xmax/width, ymax/height]   #return the four relative points

#set value to labels
labels = [to_labels(path) for path in xmls_path]
# labels[:3]

#set four labels as outputs
out1,out2,out3,out4 = list(zip(*labels))
#convert to np.array
out1 = np.array(out1)
out2 = np.array(out2)
out3 = np.array(out3)
out4 = np.array(out4)
label = np.array(names.values)

#label to tf.data
label_datasets = tf.data.Dataset.from_tensor_slices((out1,out2,out3,out4,label))

#def load_image function
def load_image(path):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image,3)
    image = tf.image.resize(image,[224,224])
    image = tf.cast(image/127.5-1,tf.float32)
    return image

#build dataset
dataset = tf.data.Dataset.from_tensor_slices(imgs_train)
dataset = dataset.map(load_image)

dataset_label = tf.data.Dataset.zip((dataset,label_datasets))

#batch constant
BATCH_SIZE = 16
AUTO = tf.data.experimental.AUTOTUNE

#batch extraction and shuffle
dataset_label = dataset_label.repeat().shuffle(500).batch(BATCH_SIZE)
dataset_label = dataset_label.prefetch(AUTO)

#Split dataset
test_count = int(len(imgs_train)*0.2)
train_count = len(imgs_train) - test_count
# test_count,train_count

train_dataset = dataset_label.skip(test_count)
test_dataset = dataset_label.take(test_count)

species_dict = {v:k for k,v in Class_dict.items()}

#check from train_data
for img, label in train_dataset.take(1):
    plt.imshow(keras.preprocessing.image.array_to_img(img[0]))
    out1,out2,out3,out4,out5 = label
    xmin,ymin,xmax,ymax = out1[0].numpy()*224,out2[0].numpy()*224,out3[0].numpy()*224,out4[0].numpy()*224
    rect = Rectangle((xmin,ymin),(xmax-xmin),(ymax-ymin),fill=False,color='r')
    ax = plt.gca()
    ax.axes.add_patch(rect)
    pred_imglist = []
    pred_imglist.append(species_dict[np.argmax(out5[0])+1])
    plt.title(pred_imglist)
    plt.show()

# 2. Model Building
# 2.1 The VGG16 model
#Convolution based
conv = keras.applications.xception.Xception(weights='imagenet',
                                            include_top=False,
                                            input_shape=(224,224,3),
                                            pooling='avg')
#open trainable
conv.trainable = True

#define Conv + FC structure
inputs = keras.Input(shape=(224,224,3))
x = conv(inputs)
x1 = keras.layers.Dense(1024,activation='relu')(x)
x1 = keras.layers.Dense(512,activation='relu')(x1)


out1 = keras.layers.Dense(1,name='out1')(x1)
out2 = keras.layers.Dense(1,name='out2')(x1)
out3 = keras.layers.Dense(1,name='out3')(x1)
out4 = keras.layers.Dense(1,name='out4')(x1)

x2 = keras.layers.Dense(1024,activation='relu')(x)
x2 = keras.layers.Dropout(0.5)(x2)
x2 = keras.layers.Dense(512,activation='relu')(x2)
out_class = keras.layers.Dense(10,activation='softmax',name='out_item')(x2)

out = [out1,out2,out3,out4,out_class]

model = keras.models.Model(inputs=inputs,outputs=out)
model.summary()

from tensorflow.keras.utils import to_categorical

# Class_dict 정의
Class_dict = {
    'crease': 1,
    'rolled_pit':2,
    'welding_line': 3,
    'water_spot': 4,
    'silk_spot': 5,
    'punching_hole': 6,
    'oil_spot': 7,
    'waist folding': 8,
    'inclusion': 9,
    'crescent_gap': 10
}

# 이미지에 대한 클래스 레이블을 Class_dict의 정수값으로 변환하여 valid_labels에 저장
image_classes =['crease','crescent_gap','oil_spot','rolled_pit','punching_hole','silk_spot','inclusion','waist folding','water_spot','welding_line']
valid_labels = [Class_dict[cls] for cls in image_classes]

print(valid_labels)  # 정수형 레이블

# 정답 레이블을 원핫 인코딩하지 않고 정수형 레이블 그대로 사용
# 모델 컴파일 (각 출력에 맞는 loss와 metrics 제공)
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.002),  # 초기 학습률 설정
              loss={
                  'out1': 'mse',
                  'out2': 'mse',
                  'out3': 'mse',
                  'out4': 'mse',
                  'out_item': 'sparse_categorical_crossentropy'  # 정수형 레이블 사용
              },
              metrics={
                  'out1': 'mae',
                  'out2': 'mae',
                  'out3': 'mae',
                  'out4': 'mae',
                  'out_item': 'accuracy'  # 정확도 사용
              })

# 학습률 감소 콜백 설정
lr_reduce = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', patience=6, factor=0.5, min_lr=1e-8)

# 모델 학습
history = model.fit(
    train_dataset,  # 학습 데이터셋
    steps_per_epoch=train_count // BATCH_SIZE,  # 학습 데이터에서 스텝 수 계산
    epochs=50,  # 에포크 수 설정
    validation_data=test_dataset,  # 검증 데이터셋
    validation_steps=test_count // BATCH_SIZE,  # 검증 데이터에서 스텝 수 계산
    callbacks=[lr_reduce]  # 학습률 감소 콜백 추가
)
#training visualization
def plot_history(history):
    hist = pd.DataFrame(history.history)
    hist['epoch']=history.epoch

    plt.figure()
    plt.xlabel('Epoch')
    plt.ylabel('MSE')
    plt.plot(hist['epoch'],hist['loss'],
            label='Train Loss')
    plt.plot(hist['epoch'],hist['val_loss'],
            label='Val Loss')
    plt.legend()

    plt.figure()
    plt.xlabel('Epoch')
    plt.ylabel('Val_MAE')
    plt.plot(hist['epoch'],hist['val_out1_mae'],
            label='Out1_mae')
    plt.plot(hist['epoch'],hist['val_out2_mae'],
            label='Out2_mae')
    plt.plot(hist['epoch'],hist['val_out3_mae'],
            label='Out3_mae')
    plt.plot(hist['epoch'],hist['val_out4_mae'],
            label='Out4_mae')
    plt.legend()

    # plt.figure()
    # plt.xlabel('Epoch')
    # plt.ylabel('Val_Item_Acc')
    plt.figure()
    plt.xlabel('Epoch')
    plt.ylabel('Val_Item_Acc')
    plt.plot(hist['epoch'], hist['val_out_item_accuracy'], label='Out5_acc')
    plt.legend()

    plt.show()

plot_history(history)

mae = model.evaluate(test_dataset)

# print('out1_mae in test:{}'.format(mae[6]))
# print('out2_mae in test:{}'.format(mae[8]))
# print('out3_mae in test:{}'.format(mae[10]))
# print('out4_mae in test:{}'.format(mae[12]))
# print('class_label in test:{}'.format(mae[15]))
print('out1_mae in test:{}'.format(mae[0]))
print('out2_mae in test:{}'.format(mae[1]))
print('out3_mae in test:{}'.format(mae[2]))
print('out4_mae in test:{}'.format(mae[3]))
print('class_label in test:{}'.format(mae[5]))  # out_item_accuracy

model.save(f"/content/drive/MyDrive/비전ai/archive/model_save/class_location_test3_{결함}.h5")
model.save(f"/content/drive/MyDrive/비전ai/archive/model_save/class_location_test3_{결함}.keras")
species_dict = {v:k for k,v in Class_dict.items()}

# plt.figure(figsize=(10,24))
# for img,_ in train_dataset.take(1):
#     out1, out2, out3, out4, out5 = model.predict(img)
#     for i in range(3):
#         plt.axis('off')
#         plt.subplot(3, 1, i+1)
#         plt.imshow(keras.preprocessing.image.array_to_img(img[i]))

#         pred_class_index = np.argmax(out5[i]) + 1  # 예측된 클래스 인덱스
#         print(f"Predicted class index: {pred_class_index}")

#         pred_imglist = []
#         # species_dict에서 예측된 클래스 인덱스가 존재하는지 확인
#         if pred_class_index in species_dict:
#             pred_imglist.append(species_dict[pred_class_index])
#         else:
#             pred_imglist.append(f"Unknown class {pred_class_index}")

#         plt.title(pred_imglist)

#         xmin, ymin, xmax, ymax = out1[i] * 224, out2[i] * 224, out3[i] * 224, out4[i] * 224
#         rect = Rectangle((xmin, ymin), (xmax - xmin), (ymax - ymin), fill=False, color='r')
#         ax = plt.gca()
#         ax.axes.add_patch(rect)
plt.figure(figsize=(10,24))

# 모델에서 예측한 출력(out5)이 클래스 예측 결과라고 가정
for img, _ in train_dataset.take(1):
    out1, out2, out3, out4, out5 = model.predict(img)

    # 3개의 이미지를 순회
    for i in range(10):
        plt.axis('off')  # 축 표시 제거
        plt.subplot(10, 1, i+1)

        # 이미지 시각화
        plt.imshow(keras.preprocessing.image.array_to_img(img[i]))

        # 예측된 클래스 인덱스
        pred_class_index = np.argmax(out5[i]) + 1  # species_dict가 1부터 시작한다면

        # 클래스가 항상 species_dict에 존재한다고 가정하고 처리
        pred_imglist = [species_dict[pred_class_index]]

        # 제목 설정
        plt.title(pred_imglist)

        # 바운딩 박스 그리기
        xmin, ymin, xmax, ymax = out1[i] * 224, out2[i] * 224, out3[i] * 224, out4[i] * 224
        rect = Rectangle((xmin, ymin), (xmax - xmin), (ymax - ymin), fill=False, color='r')
        ax = plt.gca()  # 현재의 축 가져오기
        ax.axes.add_patch(rect)

# 시각화한 이미지 출력
plt.show()
# except:
#   print(f'{i}폴더는 생성하지 못했습니다')